In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *
from textstat.textstat import textstat
import urllib, bs4

pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter("ignore", category=UserWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
outlet1 = 'dailymail.co.uk'
outlet2 = 'nytimes.com'
outlet3 = 'theatlantic.com'
outlet4 = 'washingtonpost.com'
#DailyMail -> Bad
#NYTimes -> Neutral for acsh Top for climatefeedback
#the Atlantic -> Top for acsh Neutral for climatefeedback

In [11]:
outlet = outlet3
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')


df[df.apply(lambda x: analyze_url(x['url'])[0]==outlet, axis=1)].to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+outlet+'.tsv', cache_dir+outlet+'.tsv')
df = pd.read_csv(cache_dir+outlet+'.tsv', sep='\t').fillna(0)

for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/exp2/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/exp2/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/exp2/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/exp2/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(x).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(x).polarity)

df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


df['has_author'] = ~(df['authors'].isnull() | (df['authors'] == ''))

G = read_graph(cache_dir+'diffusion_graph/pruned_graph_v3.tsv')
pr = nx.pagerank(G.reverse())

df['pageRank'] = df['url'].apply(lambda x: pr[x])

df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


df.url = df.url.apply(lambda x: analyze_url(x)[0])

df[['url', 'likes', 'replies_count',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']].to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

In [12]:
df = pd.concat([pd.read_csv(cache_dir+outlet1+'.tsv', sep='\t'), pd.read_csv(cache_dir+outlet2+'.tsv', sep='\t'), pd.read_csv(cache_dir+outlet3+'.tsv', sep='\t')])
y = df[['url']].values
X = np.array(df.drop('url', axis=1).values, dtype=np.float32)

In [ ]:
n_est = 100
m_dep = 10
X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)
classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1)
classifier.fit(X, y)
result = classifier.feature_importances_

In [13]:
from sklearn.feature_selection import f_classif
_, result = f_classif(X,y)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/opt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [14]:
d = {c:r for c, r in zip(df.columns[1:], result)}
sorted(d.items(), key=lambda kv: kv[1])

[('alexa_rank', 0.0),
 ('count_ORG_quotes', 3.5488347e-07),
 ('word_count', 0.006087065),
 ('count_all_quotes', 0.011927993),
 ('title_polarity', 0.020459978),
 ('replies_count', 0.12580314),
 ('count_unnamed_quotes', 0.12993713),
 ('tweets_mean_polarity', 0.13073453),
 ('replies_mean_subjectivity', 0.2572758),
 ('tweets_mean_subjectivity', 0.33654085),
 ('users_median_followers', 0.3874908),
 ('tweets_time_delta', 0.5226302),
 ('users_countries', 0.5748531),
 ('replies_mean_polarity', 0.5890797),
 ('pageRank', 0.600071),
 ('retweets', 0.72130746),
 ('users_median_friends', 0.7357558),
 ('stance', 0.7742688),
 ('title_subjectivity', 0.80346733),
 ('related', 0.8035073),
 ('readability', 0.8532041),
 ('likes', 0.892101),
 ('count_PER_quotes', 0.95182395),
 ('users_median_tweets', 0.99825907),
 ('has_author', nan)]